In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import pprint
import getpass
import re
from tqdm import tqdm

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

### Add genre information and release date to dataframe

#### Setup access to Spotify API

In [2]:
clientid = getpass.getpass('Enter Client ID: ')
clientsecret = getpass.getpass('Enter Client Secret: ')

Enter Client ID: ········
Enter Client Secret: ········


In [3]:
#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=clientid,
    client_secret=clientsecret))

#### Load audio features into dataframe

In [4]:
audio_feat = pd.read_pickle("../../files/pickle/audioanalysis.pkl")

In [6]:
audio_feat.head()

,id,artist,title,year,month,week,totalweeksincharts,total2018weeksincharts,total2019weeksincharts,total2020weeksincharts,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,spotify-id,track_href,analysis_url,uri
0,1,Kollegah & Farid Bang,One Night Stand,2018,1,1,4,4,0,0,...,0.000000,0.1670,0.309,108.883,163773,4,7J82QBqJjwbIDtm1Oxc0uo,https://api.spotify.com/v1/tracks/7J82QBqJjwbI...,https://api.spotify.com/v1/audio-analysis/7J82...,spotify:track:7J82QBqJjwbIDtm1Oxc0uo
1,2,Bausa,FML,2018,1,1,7,7,0,0,...,0.000309,0.1640,0.246,108.589,179718,4,4OwriBNZjK7VtYBxngT3OJ,https://api.spotify.com/v1/tracks/4OwriBNZjK7V...,https://api.spotify.com/v1/audio-analysis/4Owr...,spotify:track:4OwriBNZjK7VtYBxngT3OJ
2,3,Jazn,Bombay,2018,1,1,9,9,0,0,...,0.000167,0.0994,0.422,96.032,193040,4,083GGYAqJqUkRsy8JyLeyG,https://api.spotify.com/v1/tracks/083GGYAqJqUk...,https://api.spotify.com/v1/audio-analysis/083G...,spotify:track:083GGYAqJqUkRsy8JyLeyG
3,4,Tom Walker,Leave A Light On,2018,1,1,28,28,0,0,...,0.000002,0.1330,0.267,68.976,185863,4,6lOWoTqVnAWXchddtTH31W,https://api.spotify.com/v1/tracks/6lOWoTqVnAWX...,https://api.spotify.com/v1/audio-analysis/6lOW...,spotify:track:6lOWoTqVnAWXchddtTH31W
4,5,G-Eazy feat. A$AP Rocky & Cardi B,No Limit,2018,1,1,5,5,0,0,...,0.000000,0.0853,0.405,175.957,245387,4,2DQ1ITjI0YoLFzuADN1ZBW,https://api.spotify.com/v1/tracks/2DQ1ITjI0YoL...,https://api.spotify.com/v1/audio-analysis/2DQ1...,spotify:track:2DQ1ITjI0YoLFzuADN1ZBW


#### Find information about genres and release date and add them to dataframe

In [7]:
genres = []
release_date = []

for spotify_id in tqdm(audio_feat['spotify-id']):
    track = sp.track(spotify_id)
    #find the genre of the artist
    artist = sp.artist(track["artists"][0]["external_urls"]["spotify"])
    genres.append(artist["genres"])
    #find the release_date of the album with which the respective song was released
    album = sp.album(track["album"]["external_urls"]["spotify"])
    release_date.append(album["release_date"])
    
audio_feat['genres'] = genres
audio_feat['release'] = release_date

100%|██████████████████████████████████████████████████████████████████████████████| 1897/1897 [08:56<00:00,  3.53it/s]


#### Create additional column with only the first entry in every genres list of every song and save dataframe

In [8]:
genre = []
for g in audio_feat['genres']:
    if len(g) > 0:
        gen = g[0]
        genre.append(gen)
    else:
        gen = np.nan
        genre.append(gen)
genre

audio_feat['genre'] = genre

In [9]:
audio_feat['genre'].value_counts()

german hip hop         522
deep german hip hop    213
dance pop              159
german pop              97
albanian hip hop        92
                      ... 
german oi                1
dark clubbing            1
dutch house              1
beatlesque               1
viral rap                1
Name: genre, Length: 137, dtype: int64

In [10]:
audio_feat.to_pickle("../../files/pickle/audioanalysis_compl.pkl")
audio_feat.to_csv("../../files/csv/audioanalysis_compl.csv")